## Python para Investimentos -  Analisando Fundos de Investimento com Python

*   Base de Dados: http://dados.cvm.gov.br/




## Módulos

In [ ]:
!pip install -U plotly

     |████████████████████████████████| 20.6MB 1.5MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
!pip install -q yfinance

     |████████████████████████████████| 6.3MB 8.8MB/s 


In [ ]:
#Cotações do Yahoo Finance
import yfinance as yf

# Tratamento de Dados
import pandas as pd
from pandas.tseries.offsets import BDay
pd.set_option("display.max_colwidth", 150)

#Gráficos
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

#Utilidades
from datetime import date
import IPython
from google.colab.data_table import DataTable

## Funções

In [ ]:
def busca_cadastro_cvm(): 
  try:
    url = 'http://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv'
    return pd.read_csv(url, sep=';', encoding='ISO-8859-1')
  except: 
    print("Arquivo {} não encontrado!".format(url))

In [ ]:
def busca_informes_diarios_cvm_por_periodo(data_inicio, data_fim):
  datas = pd.date_range(data_inicio, data_fim, freq='MS') 
  informe_completo = pd.DataFrame()

  for data in datas:
    try:
      url ='http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{}{:02d}.csv'.format(data.year, data.month)
      informe_mensal = pd.read_csv(url, sep=';')    
    
    except: 
      print("Arquivo {} não encontrado!".format(url))    

    informe_completo = pd.concat([informe_completo, informe_mensal], ignore_index=True)

  return informe_completo

In [ ]:
def consulta_fundo(informes, cnpj):  
  fundo = informes[informes['CNPJ_FUNDO'] == cnpj].copy()
  fundo.set_index('DT_COMPTC', inplace=True)
  fundo['cotas_normalizadas'] = (fundo['VL_QUOTA'] / fundo['VL_QUOTA'].iloc[0])*100
  return fundo

In [ ]:
def ranking_fundos(informes, cadastro, minimo_de_cotistas=100, classe=''):    
  '''
    Retorna uma lista ordenada com o retorno dos fundos no período  
    Argumentos:
      informes: retorno da função busca_informes_diarios_cvm_por_periodo() 
      cadastro: retorno da função busca_cadastro_cvm()
      minimo_de_cotistas: filtra fundos com o mínimo de cotistas
      classe: 'Fundo de Ações', 'Fundo Multimercado', 'Fundo de Renda Fixa', 'Fundo Cambial'
      
  '''
  
  cadastro      = cadastro[cadastro['SIT'] == 'EM FUNCIONAMENTO NORMAL']
  fundos        = informes[informes['NR_COTST'] >= minimo_de_cotistas]
  cnpj_informes = fundos['CNPJ_FUNDO'].drop_duplicates()
  
  fundos = fundos.pivot(index='DT_COMPTC', columns='CNPJ_FUNDO')
  retorno = fundos['VL_QUOTA'].pct_change().mean()*252
  vol     = fundos['VL_QUOTA'].pct_change().std()*(252**0.5)
  cotas_normalizadas = fundos['VL_QUOTA'] / fundos['VL_QUOTA'].iloc[0]

  try:
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == classe]['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]
  except:
    print("Parâmetro 'classe' informado incorretamente!")
    print("As opções disponíveis são:",cadastro['CLASSE'].unique())
    return
  
  performance_fundos = pd.DataFrame()
  performance_fundos['retorno(%)'] = round((cotas_normalizadas.iloc[-1].sort_values(ascending=False) - 1)  * 100, 2)
  for cnpj in performance_fundos.index:
    fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
    performance_fundos.at[cnpj, 'retorno_anualizado(%)'] = round(retorno.loc[cnpj]*100, 2)
    performance_fundos.at[cnpj, 'vol_anualizada(%)']     = round(vol.loc[cnpj]*100, 2)
    performance_fundos.at[cnpj, 'retorno/risco']         = round((retorno / vol).loc[cnpj], 2)
    performance_fundos.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
    performance_fundos.at[cnpj, 'Classe']                = fundo['CLASSE'].values[0]
    performance_fundos.at[cnpj, 'PL']                    = fundo['VL_PATRIM_LIQ'].values[0]
                                                    
  
  return performance_fundos.dropna()

In [ ]:
def cdi_acumulado(data_inicio, data_fim):
  codigo_bcb = 12
  
  url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
  cdi = pd.read_json(url)
  cdi['data'] = pd.to_datetime(cdi['data'], dayfirst=True)
  cdi.set_index('data', inplace=True) 
  
  cdi_acumulado = (1 + cdi[data_inicio : data_fim] / 100).cumprod()
  cdi_acumulado.iloc[0] = 1
  return cdi_acumulado

## Benckmarks

In [ ]:
data_inicio="2020-12-30"
data_fim="2021-06-30"

In [ ]:
cdi = cdi_acumulado(data_inicio, data_fim) * 100
cdi

,valor
data,
2020-12-30,100.000000
2020-12-31,100.014939
2021-01-04,100.022409
2021-01-05,100.029879
2021-01-06,100.037351
...,...
2021-06-24,101.227392
2021-06-25,101.243728
2021-06-28,101.260065


In [ ]:
ibov = yf.download('^BVSP', start=data_inicio, end=data_fim)['Adj Close']

[*********************100%***********************]  1 of 1 completed


In [ ]:
ibov_ret_total = ((ibov.iloc[-1] / ibov.iloc[0]) - 1 ) *  100
cdi_ret_total = (cdi.iloc[-1] - 100)[0]
ibov_ret = ibov.pct_change().mean()*252*100
ibov_vol = ibov.pct_change().std()*(252**0.5)*100

## Buscando Dados da CVM

In [ ]:
cadastro = busca_cadastro_cvm()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (14,17,18,20,22,24,27,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
informes = busca_informes_diarios_cvm_por_periodo(data_inicio=data_inicio, data_fim=data_fim)

## Resultados

In [ ]:
# classe: 'Fundo de Ações', 'Fundo Multimercado', 'Fundo de Renda Fixa', 'Fundo Cambial'
fundos = ranking_fundos(informes, cadastro, minimo_de_cotistas=500, classe='Fundo de Ações')

In [ ]:
fundos["retorno(%)"].describe()

count    449.000000
mean       8.811069
std        8.433968
min      -20.480000
25%        4.590000
50%        6.840000
75%       11.100000
max       54.690000
Name: retorno(%), dtype: float64

In [ ]:
superou_ibov = (fundos["retorno(%)"] > ibov_ret_total).sum() / fundos.shape[0]
superou_cdi  = (fundos["retorno(%)"] > cdi_ret_total).sum() / fundos.shape[0]
print(f"{superou_ibov:.2%} dos fundos superou o Ínbice Bovespa")
print(f"{superou_cdi:.2%} dos fundos superou o CDI")

50.33% dos fundos superou o Ínbice Bovespa
92.43% dos fundos superou o CDI


In [ ]:
DataTable(fundos)

,retorno(%),retorno_anualizado(%),vol_anualizada(%),retorno/risco,Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,,,,
35.471.498/0001-55,54.69,91.88,17.98,5.11,TRIGONO FLAGSHIP 60 SMALL CAPS FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM ACOES,Fundo de Ações,3.542686e+08
29.177.013/0001-12,54.64,91.84,18.12,5.07,TRIGONO FLAGSHIP SMALL CAPS FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM ACOES,Fundo de Ações,2.679313e+08
08.968.733/0001-26,52.15,88.11,15.97,5.52,FUNDO DE INVESTIMENTO DE AÇÕES TRIGONO VERBIER,Fundo de Ações,2.247656e+08
29.177.024/0001-00,51.25,87.08,17.19,5.07,TRIGONO DELPHOS INCOME FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM ACOES,Fundo de Ações,1.953756e+08
24.874.367/0001-00,36.55,70.51,34.88,2.02,INTER + IBOVESPA ATIVO FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,3.464518e+08
...,...,...,...,...,...,...,...
10.590.125/0001-72,-5.87,-3.95,41.87,-0.09,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES CIELO,Fundo de Ações,8.380154e+06
10.869.628/0001-81,-6.59,-5.59,41.68,-0.13,BB AÇÕES CIELO FUNDO DE INVESTIMENTO,Fundo de Ações,2.190263e+07
30.068.271/0001-40,-11.87,-20.53,33.16,-0.62,CAIXA FUNDO DE INVESTIMENTO EM AÇÕES BANCO DO BRASIL PLUS,Fundo de Ações,1.295855e+08


In [ ]:
fig = px.histogram(x=fundos['retorno(%)'], title="Histograma - Retorno dos Fundos", nbins=100,labels =dict(x="retorno(%)", y="Ocorrencias"))
fig.add_vline(ibov_ret_total, line_color="red", annotation_text =f"IBOV ({ibov_ret_total:.2}%)")
fig.add_vline(cdi_ret_total, line_color="blue", annotation_text =f"CDI ({cdi_ret_total:.2}%)")
fig.update_layout(bargap=0.01, showlegend=False)

In [ ]:
fig = px.scatter(data_frame=fundos, 
           y="retorno_anualizado(%)", 
           x="vol_anualizada(%)", 
           color='retorno/risco',
           hover_name=fundos['Fundo de Investimento']
           )

fig.add_scatter(x=[ibov_vol], y=[ibov_ret], line_color='black', showlegend=False, name="IBOV")
fig.add_vline(ibov_vol )
fig.add_hline(ibov_ret)